In [1]:
import pandas as pd
import numpy as np
from binance.client import Client
import binance_keys as bk
import datetime as dt
from IPython.display import clear_output
from datetime import timedelta
import time
import candles_getting as candles
import plotly.express as px

In [2]:
client = Client(bk.api_key, bk.secret_key)


In [3]:
all_tickers = pd.DataFrame(client.get_ticker())

In [4]:
all_tickers[['priceChange', 'priceChangePercent', 'weightedAvgPrice',
       'prevClosePrice', 'lastPrice', 'lastQty', 'bidPrice', 'bidQty',
       'askPrice', 'askQty', 'openPrice', 'highPrice', 'lowPrice', 'volume',
       'quoteVolume']] = all_tickers[['priceChange', 'priceChangePercent', 'weightedAvgPrice',
       'prevClosePrice', 'lastPrice', 'lastQty', 'bidPrice', 'bidQty',
       'askPrice', 'askQty', 'openPrice', 'highPrice', 'lowPrice', 'volume',
       'quoteVolume']].astype('float').sort_values('quoteVolume', ascending = False)

In [5]:
usdt_pairs_info = all_tickers[all_tickers['symbol'].str.endswith('USDT')].head(10)['symbol'].to_list()
btc_pairs_info = all_tickers[all_tickers['symbol'].str.endswith('BTC')].head(10)['symbol'].to_list()
eth_pairs_info = all_tickers[all_tickers['symbol'].str.endswith('ETH')].head(10)['symbol'].to_list()

In [6]:
full_list = usdt_pairs_info + btc_pairs_info + eth_pairs_info

In [7]:
common_df_lst = []
cnt = 0

for i in full_list:
    
    print(i, 'in progress')
    
    try:
        
        df = candles.hist_prices(i, '1h', '2022-01-01', '2022-12-27')
        df['symbol'] = i
        df['open_time_hr'] = df['open_time'].dt.hour
        df['week'] = df['open_time'].dt.week
        df['close_shift1'] =  df['close'].pct_change() * 100
        common_df_lst.append(df)
        
    except:
        
        print('Error with {i}')
        
    cnt+=1
    clear_output(wait = True)

WTCETH in progress
Error with {i}


In [17]:
common_df = pd.concat(common_df_lst)

In [18]:
common_df['period_range'] = common_df['high'] - common_df['low']

In [19]:
week_stats = common_df.groupby(['symbol','week','open_time_hr'])['period_range'].median().reset_index()

In [20]:
hour_stats = common_df.groupby(['symbol','open_time_hr'])['period_range'].median().reset_index()

In [21]:
indx_lst = hour_stats.groupby(['symbol'])['period_range'].nlargest(3).reset_index()['level_1']

In [22]:
top3_data = hour_stats.iloc[indx_lst]

In [23]:
px.bar(hour_stats.query('symbol == "BTCUSDT"'), x = 'open_time_hr', y = 'period_range')

In [26]:
fig = px.line(week_stats.query('symbol == "BTCUSDT"'), x = 'week', y = 'period_range', color = 'open_time_hr')
fig.update_layout(template='plotly_white')
fig.show()


In [16]:
top3_data

,symbol,open_time_hr,period_range
0,ADAUSDT,0,9.100000e-03
14,ADAUSDT,14,8.950000e-03
13,ADAUSDT,13,8.700000e-03
24,BNBBTC,0,6.900000e-05
39,BNBBTC,15,6.700000e-05
...,...,...,...
470,WTCBTC,14,1.400000e-07
464,WTCBTC,8,1.400000e-07
480,XRPUSDT,0,6.500000e-03
494,XRPUSDT,14,6.200000e-03
